# Exercices
1) Envoyer le fichier persons.json dans un topic Kafka
2) Lire le topic en streaming avec Spark
3) Stocker les données en DB dans une table raw_data
4) Lecture en batch des données depuis la table raw_data
5) Parser les données JSON
6) Stocker les données nettoyées dans une table persons_data
7) Filtrer les persones de plus de 30 ans
8) Compter le nombre de personnes par ville
9) Ajouter une colonne age_group (young -18, adult 18-64, senior 65+) et les afficher
10) Stocker les données nettoyées dans une table persons
11) Grouper les données par villes et age_group et les afficher le total de chaque groupe
12) Stocker les données agrégées dans une table persons_aggregated

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, from_json, when, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
import json
from kafka import KafkaProducer

In [2]:
kafka_boot = 'kafka:9092'
topic = 'exercice1'
postgres_data = {
    'url': 'jdbc:postgresql://postgres:5432/events',
    'username': 'app',
    'password': '1234',
    'driver': 'org.postgresql.Driver',
    'database': 'events'
}

In [3]:
producer = KafkaProducer(bootstrap_servers=kafka_boot, value_serializer=lambda v: json.dumps(v).encode('utf-8'))

with open('./data/persons.json') as f:
    data = json.load(f)

    for row in data:
        producer.send(topic, value=row)

producer.flush()
producer.close()
print("✅ Données envoyées dans Kafka.")

✅ Données envoyées dans Kafka.


In [4]:
spark = SparkSession.builder.appName('Exercice1').getOrCreate()

# Lecture du flux Kafka en Streaming Pour le traitement continu
# df = spark.readStream.format('kafka') \
#     .option('kafka.bootstrap.servers', kafka_boot) \
#     .option('subscribe', topic) \
#     .option('startingOffsets', 'earliest') \
#     .load()

# Lecture du flux Kafka en Batch
df = spark.read.format('kafka') \
    .option('kafka.bootstrap.servers', kafka_boot) \
    .option('subscribe', topic) \
    .option('startingOffsets', 'earliest') \
    .load()

df_parsed = df.selectExpr("CAST(value AS STRING) AS raw")
# Vu que je suis en batch je peux faire un show
df_parsed.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/08 09:49:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+
|                 raw|
+--------------------+
|{"id": 1, "name":...|
|{"id": 2, "name":...|
|{"id": 3, "name":...|
|{"id": 4, "name":...|
|{"id": 5, "name":...|
|{"id": 6, "name":...|
|{"id": 7, "name":...|
|{"id": 8, "name":...|
|{"id": 9, "name":...|
|{"id": 10, "name"...|
|{"id": 11, "name"...|
|{"id": 12, "name"...|
|{"id": 13, "name"...|
|{"id": 14, "name"...|
|{"id": 15, "name"...|
|{"id": 16, "name"...|
|{"id": 17, "name"...|
|{"id": 18, "name"...|
|{"id": 19, "name"...|
|{"id": 20, "name"...|
+--------------------+
only showing top 20 rows



In [5]:
df.write.format('jdbc') \
    .option('url', postgres_data['url']) \
    .option('dbtable', 'public.raw_data') \
    .option('user', postgres_data['username']) \
    .option('password', postgres_data['password']) \
    .mode('append') \
    .save()

print("✅ Données brutes stockées dans la table raw_data.")

✅ Données brutes stockées dans la table raw_data.


In [6]:
df_silver = spark.read.format('jdbc') \
    .option('url', postgres_data['url']) \
    .option('dbtable', 'public.raw_data') \
    .option('user', postgres_data['username']) \
    .option('password', postgres_data['password']) \
    .option('driver', postgres_data['driver']) \
    .load()

df_silver.show(10, truncate=False)

+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------+------+-----------------------+-------------+
|key |value                                                                                                                                                                                            |topic    |partition|offset|timestamp              |timestampType|
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------+------+-----------------------+-------------+
|null|[7B 22 69 64 22 3A 20 31 2C 20 22 6E 61 6D 65 22 3A 20 22 41 6C 69 63 65 22 2C 20 22 61 67 65 22 3A 20 33 30 2C 20 22 63 69 74 79 22 3A 20 22 50 61 72 69 73 22 7D]                              |ex

In [9]:
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), False),
    StructField("city", StringType(), False)
])

df_silver = df_silver.select(from_json(col('value').cast('string'), schema).alias('data')).select('data.*')
df_silver.show(10, truncate=False)

+---+---------+---+----------+
|id |name     |age|city      |
+---+---------+---+----------+
|1  |Alice    |30 |Paris     |
|2  |Bob      |25 |Lyon      |
|3  |Céline   |35 |Marseille |
|4  |David    |28 |Paris     |
|5  |Emma     |40 |Bordeaux  |
|6  |François |22 |Nice      |
|7  |Gabrielle|31 |Strasbourg|
|8  |Hugo     |27 |Lille     |
|9  |Inès     |29 |Nantes    |
|10 |Julien   |33 |Toulouse  |
+---+---------+---+----------+
only showing top 10 rows



In [10]:
df_silver.write.format('jdbc') \
    .option('url', postgres_data['url']) \
    .option('dbtable', 'public.persons_data') \
    .option('user', postgres_data['username']) \
    .option('password', postgres_data['password']) \
    .mode('append') \
    .save()

In [12]:
df_silver.filter(col('age') > 30).show()

+---+---------+---+-----------+
| id|     name|age|       city|
+---+---------+---+-----------+
|  3|   Céline| 35|  Marseille|
|  5|     Emma| 40|   Bordeaux|
|  7|Gabrielle| 31| Strasbourg|
| 10|   Julien| 33|   Toulouse|
| 11|    Karim| 41|Montpellier|
| 12|    Laura| 36|      Paris|
| 15|  Olivier| 39|   Bordeaux|
| 16|  Pauline| 34|       Nice|
| 18|    Rania| 32|      Lille|
| 19|   Sophie| 38|     Nantes|
| 23|  William| 37|       Lyon|
| 24|   Xavier| 45|  Marseille|
| 27|   Adrien| 33| Strasbourg|
| 28| Brigitte| 42|      Lille|
| 30| Delphine| 31|   Toulouse|
| 31|     Éric| 40|Montpellier|
| 33|  Georges| 36|       Lyon|
| 36|    Julie| 34|       Nice|
| 37|    Kevin| 39| Strasbourg|
| 40|    Nadia| 37|   Toulouse|
+---+---------+---+-----------+
only showing top 20 rows



In [13]:
df_silver.groupBy('city').count().show()

+-----------+-----+
|       city|count|
+-----------+-----+
|       Nice|    5|
|Montpellier|    4|
|      Lille|    5|
|     Nantes|    5|
|  Marseille|    5|
|      Paris|    6|
|       Lyon|    5|
|   Bordeaux|    5|
| Strasbourg|    5|
|   Toulouse|    5|
+-----------+-----+



In [16]:
df_silver = df_silver.withColumn(
    'age_group',
    when(col('age') < 30, lit('young < 30'))
    .when(col('age').between(30, 50), lit('adult 30-50'))
    .otherwise('senior > 50')
)

df_silver.show()

+---+---------+---+-----------+-----------+
| id|     name|age|       city|  age_group|
+---+---------+---+-----------+-----------+
|  1|    Alice| 30|      Paris|adult 30-50|
|  2|      Bob| 25|       Lyon| young < 30|
|  3|   Céline| 35|  Marseille|adult 30-50|
|  4|    David| 28|      Paris| young < 30|
|  5|     Emma| 40|   Bordeaux|adult 30-50|
|  6| François| 22|       Nice| young < 30|
|  7|Gabrielle| 31| Strasbourg|adult 30-50|
|  8|     Hugo| 27|      Lille| young < 30|
|  9|     Inès| 29|     Nantes| young < 30|
| 10|   Julien| 33|   Toulouse|adult 30-50|
| 11|    Karim| 41|Montpellier|adult 30-50|
| 12|    Laura| 36|      Paris|adult 30-50|
| 13|  Mathieu| 24|       Lyon| young < 30|
| 14|     Nina| 26|  Marseille| young < 30|
| 15|  Olivier| 39|   Bordeaux|adult 30-50|
| 16|  Pauline| 34|       Nice|adult 30-50|
| 17|  Quentin| 23| Strasbourg| young < 30|
| 18|    Rania| 32|      Lille|adult 30-50|
| 19|   Sophie| 38|     Nantes|adult 30-50|
| 20|   Thomas| 27|   Toulouse| 

In [17]:
df_silver.write.format('jdbc') \
    .option('url', postgres_data['url']) \
    .option('dbtable', 'public.persons_processed') \
    .option('user', postgres_data['username']) \
    .option('password', postgres_data['password']) \
    .mode('append') \
    .save()

In [19]:
df_gold = df_silver.groupBy('city', 'age_group').count()
df_gold.show()

+-----------+-----------+-----+
|       city|  age_group|count|
+-----------+-----------+-----+
| Strasbourg| young < 30|    1|
|  Marseille| young < 30|    3|
|       Lyon|adult 30-50|    2|
|     Nantes|adult 30-50|    2|
|       Nice|adult 30-50|    2|
|      Paris|adult 30-50|    3|
|   Toulouse| young < 30|    2|
|  Marseille|adult 30-50|    2|
|   Bordeaux|adult 30-50|    3|
|Montpellier|adult 30-50|    4|
|      Lille| young < 30|    2|
|   Toulouse|adult 30-50|    3|
|     Nantes| young < 30|    3|
|      Paris| young < 30|    3|
| Strasbourg|adult 30-50|    4|
|   Bordeaux| young < 30|    2|
|       Nice| young < 30|    3|
|       Lyon| young < 30|    3|
|      Lille|adult 30-50|    3|
+-----------+-----------+-----+



In [20]:
df_gold.write.format('jdbc') \
    .option('url', postgres_data['url']) \
    .option('dbtable', 'public.persons_aggregated') \
    .option('user', postgres_data['username']) \
    .option('password', postgres_data['password']) \
    .mode('append') \
    .save()